# Kernel Operations & Entanglement

This notebook explores the Kernel as a stateless transformation engine and demonstrates entanglement concepts.

**Key Concepts**:
- Pure morphisms (stateless transformations)
- Multi-level operations (basic sets, entanglement, network)
- Isomorphism detection
- Singularity and coherence

## 1. Kernel: Stateless Transformations

The Kernel provides pure operations - same input always produces same output.

In [1]:
from core import Kernel, HLLSet

# Create kernel
kernel = Kernel()

print("Kernel is stateless:")
print("  - No storage")
print("  - No history")
print("  - No CAS (Content-Addressable Storage)")
print("  - Pure transformations only")
print(f"\nKernel instance: {kernel}")

Kernel is stateless:
  - No storage
  - No history
  - No CAS (Content-Addressable Storage)
  - Pure transformations only

Kernel instance: <core.kernel.Kernel object at 0x7f4db45620c0>


## 2. Level 1: Basic Set Operations

Core morphisms for HLLSet manipulation.

In [2]:
# Absorb: tokens → HLLSet
hll_a = kernel.absorb(['apple', 'banana', 'cherry'])
hll_b = kernel.absorb(['banana', 'cherry', 'date'])

print("Basic Operations:\n")
print(f"A: {hll_a}")
print(f"B: {hll_b}")

# Union: A ∪ B
union = kernel.union(hll_a, hll_b)
print(f"\nA ∪ B: {union}")

# Intersection: A ∩ B
intersection = kernel.intersection(hll_a, hll_b)
print(f"A ∩ B: {intersection}")

# Difference: A - B
difference = kernel.difference(hll_a, hll_b)
print(f"A - B: {difference}")

Basic Operations:

A: HLLSet(983bcf39..., |A|≈4.0, backend=C/Cython)
B: HLLSet(9b080fbe..., |A|≈4.0, backend=C/Cython)

A ∪ B: HLLSet(eddbc507..., |A|≈5.0, backend=C/Cython)
A ∩ B: HLLSet(ef34c2bc..., |A|≈3.0, backend=C/Cython)
A - B: HLLSet(cede032c..., |A|≈2.0, backend=C/Cython)


## 3. Similarity Measurements

Compute similarity between HLLSets.

In [3]:
# Jaccard similarity
jaccard = kernel.similarity(hll_a, hll_b)
print(f"Jaccard similarity: {jaccard:.2%}")
print(f"  |A ∩ B| / |A ∪ B|")

# Cosine similarity (not in base Kernel, but in HLLSet)
cosine = hll_a.cosine(hll_b)
print(f"\nCosine similarity: {cosine:.2%}")

# Create more similar sets
hll_x = kernel.absorb(['a', 'b', 'c'])
hll_y = kernel.absorb(['a', 'b', 'd'])
sim_xy = kernel.similarity(hll_x, hll_y)
print(f"\nX vs Y similarity: {sim_xy:.2%} (2 common / 4 total)")

Jaccard similarity: 60.00%
  |A ∩ B| / |A ∪ B|

Cosine similarity: 75.00%

X vs Y similarity: 40.00% (2 common / 4 total)


## 4. Level 2: Fold Operations

Aggregate multiple HLLSets via union or intersection.

In [4]:
# Multiple HLLSets
hll_list = [
    kernel.absorb(['a', 'b', 'c']),
    kernel.absorb(['b', 'c', 'd']),
    kernel.absorb(['c', 'd', 'e']),
    kernel.absorb(['d', 'e', 'f'])
]

print(f"Created {len(hll_list)} HLLSets\n")

# Fold union: combine all via union
union_all = kernel.fold_union(hll_list)
print(f"Union of all: {union_all}")
print(f"  Cardinality: {union_all.cardinality():.2f}")
print(f"  Expected: ~6 (a, b, c, d, e, f)")

# Fold intersection: common to all
intersection_all = kernel.fold_intersection(hll_list)
print(f"\nIntersection of all: {intersection_all}")
print(f"  Cardinality: {intersection_all.cardinality():.2f}")

Created 4 HLLSets

Union of all: HLLSet(f70d249e..., |A|≈8.0, backend=C/Cython)
  Cardinality: 8.00
  Expected: ~6 (a, b, c, d, e, f)

Intersection of all: HLLSet(1ceaf73d..., |A|≈0.0, backend=C/Cython)
  Cardinality: 0.00


## 5. ε-Isomorphism Detection (HLLSet Level)

Find approximate isomorphisms (ε-isomorphisms) between individual HLLSets.

**Note**: This is HLLSet-level similarity (non-empty intersection), NOT true entanglement.
True entanglement is defined between **lattices of HLLSets**, comparing their **structures** (graph topology), not their content.

In [5]:
# Create two installations (different data, similar structure)
installation_1 = kernel.absorb([f'token_{i}' for i in range(100)])
installation_2 = kernel.absorb([f'token_{i}' for i in range(95, 205)])

print(f"Installation 1: {installation_1.short_name}..., |A|≈{installation_1.cardinality():.0f}")
print(f"Installation 2: {installation_2.short_name}..., |A|≈{installation_2.cardinality():.0f}")

# Find isomorphism
epsilon = 0.05  # 5% tolerance
morphism = kernel.find_isomorphism(installation_1, installation_2, epsilon=epsilon)

if morphism:
    print(f"\nFound ε-isomorphism (ε={epsilon}):")
    print(f"  Source: {morphism.source_hash}")
    print(f"  Target: {morphism.target_hash}")
    print(f"  Similarity: {morphism.similarity:.2%}")
    print(f"  Is isomorphic: {morphism.is_isomorphism}")
else:
    print(f"\nNo ε-isomorphism found (similarity too low)")

Installation 1: fbd460b3..., |A|≈106
Installation 2: f1abc9c2..., |A|≈109

Found ε-isomorphism (ε=0.05):
  Source: fbd460b3e13faf38184bce01a4d5b7b9481b9107
  Target: f1abc9c253e7e8e5b4c8d1434f34f89901f23964
  Similarity: 4.33%
  Is isomorphic: True


## 6. True Entanglement: Lattice Structural Isomorphism

**Key Insight**: Entanglement is NOT about matching HLLSets—it's about matching **structures** of **lattices**.

- **Nodes are irrelevant**: The actual HLLSets (content) don't matter
- **Structure matters**: Degree distributions, graph topology, morphism patterns
- **ε-isomorphism of lattices**: Two lattices are entangled if their structures are similar

This implements the theoretical concept:
```
∃ ϕ_ij : L(S_i) → L(S_j) such that:
1. ϕ_ij is ε-isomorphism of HLLSet lattices
2. The isomorphism preserves structural relationships
```

In [6]:
# Reload modules if kernel was modified (only needed during development)
import importlib
import core.kernel, core
importlib.reload(core.kernel)
importlib.reload(core)

from core import Kernel, LatticeMorphism
kernel = Kernel()
print("✓ Modules reloaded with new LatticeMorphism class")

✓ Modules reloaded with new LatticeMorphism class


In [7]:
from core import HRTConfig, HLLSetLattice, LatticeMorphism

# Create HRTConfig for lattices (small dimension for demo)
config = HRTConfig(p_bits=4, h_bits=8, tau=0.7, rho=0.3, epsilon=0.1)
print(f"Lattice config: dimension={config.dimension}, epsilon={config.epsilon}")

# Create three lattices with SIMILAR STRUCTURE but DIFFERENT content
# Structure = degree pattern (number of tokens absorbed at each position)
# Content = the actual tokens (which differ completely)

lattice_1 = HLLSetLattice.empty(config)
lattice_2 = HLLSetLattice.empty(config)
lattice_3 = HLLSetLattice.empty(config)

# Create SAME degree pattern for all three lattices
# Pattern: position i gets (i+1)*3 tokens → same cardinality structure
num_positions = min(10, config.dimension)

for i in range(num_positions):
    num_tokens = (i + 1) * 3  # Same pattern for all lattices
    
    # Each lattice gets DIFFERENT tokens but SAME count
    tokens_1 = {f'alpha_{i}_{j}' for j in range(num_tokens)}
    tokens_2 = {f'beta_{i}_{j}' for j in range(num_tokens)}
    tokens_3 = {f'gamma_{i}_{j}' for j in range(num_tokens)}
    
    hll_1 = kernel.absorb(tokens_1)
    hll_2 = kernel.absorb(tokens_2)
    hll_3 = kernel.absorb(tokens_3)
    
    lattice_1 = lattice_1.with_row_basic(i, hll_1)
    lattice_2 = lattice_2.with_row_basic(i, hll_2)
    lattice_3 = lattice_3.with_row_basic(i, hll_3)

print(f"\nCreated 3 lattices with DIFFERENT tokens but IDENTICAL degree structure:")
print(f"  Position 0: 3 tokens each (alpha_0_*, beta_0_*, gamma_0_*)")
print(f"  Position 1: 6 tokens each")
print(f"  ...") 
print(f"  Position {num_positions-1}: {num_positions*3} tokens each")

# Show cardinalities match
print(f"\nCardinality structure (first 5 positions):")
for i in range(min(5, num_positions)):
    card_1 = lattice_1.row_basic[i].hllset.cardinality()
    card_2 = lattice_2.row_basic[i].hllset.cardinality()
    card_3 = lattice_3.row_basic[i].hllset.cardinality()
    print(f"  Position {i}: L1={card_1:.1f}, L2={card_2:.1f}, L3={card_3:.1f}")

# Direct structural comparison using cardinalities
def compute_cardinality_correlation(lat_a: HLLSetLattice, lat_b: HLLSetLattice) -> float:
    """Compute correlation of cardinality sequences (structural similarity)."""
    cards_a = [b.hllset.cardinality() for b in lat_a.row_basic[:num_positions]]
    cards_b = [b.hllset.cardinality() for b in lat_b.row_basic[:num_positions]]
    
    n = len(cards_a)
    mean_a = sum(cards_a) / n
    mean_b = sum(cards_b) / n
    
    numerator = sum((a - mean_a) * (b - mean_b) for a, b in zip(cards_a, cards_b))
    var_a = sum((a - mean_a) ** 2 for a in cards_a)
    var_b = sum((b - mean_b) ** 2 for b in cards_b)
    
    if var_a == 0 or var_b == 0:
        return 1.0 if var_a == var_b else 0.0
    
    return numerator / ((var_a * var_b) ** 0.5)

corr_12 = compute_cardinality_correlation(lattice_1, lattice_2)
corr_13 = compute_cardinality_correlation(lattice_1, lattice_3)
corr_23 = compute_cardinality_correlation(lattice_2, lattice_3)

print(f"\nLattice STRUCTURAL similarity (cardinality correlation):")
print(f"  L1 ↔ L2: {corr_12:.4f} (should be ~1.0 - identical structure)")
print(f"  L1 ↔ L3: {corr_13:.4f}")
print(f"  L2 ↔ L3: {corr_23:.4f}")

# Show that tokens have NO overlap
hll_1_sample = lattice_1.row_basic[0].hllset
hll_2_sample = lattice_2.row_basic[0].hllset
intersection_card = hll_1_sample.intersect(hll_2_sample).cardinality()
print(f"\nToken overlap between L1 and L2 (position 0):")
print(f"  Intersection cardinality: {intersection_card:.1f} (≈0 = no overlap)")

print(f"\n" + "="*60)
print("KEY INSIGHT:")
print("="*60)
print(f"• Lattices have IDENTICAL structure (cardinality patterns)")
print(f"• Lattices have ZERO token overlap (different content)")
print(f"• ENTANGLEMENT = structural similarity, NOT content overlap")
print(f"• This is the foundation of cross-modal comparison")

Lattice config: dimension=130, epsilon=0.1

Created 3 lattices with DIFFERENT tokens but IDENTICAL degree structure:
  Position 0: 3 tokens each (alpha_0_*, beta_0_*, gamma_0_*)
  Position 1: 6 tokens each
  ...
  Position 9: 30 tokens each

Cardinality structure (first 5 positions):
  Position 0: L1=4.0, L2=4.0, L3=4.0
  Position 1: L1=6.0, L2=7.0, L3=6.0
  Position 2: L1=9.0, L2=11.0, L3=10.0
  Position 3: L1=13.0, L2=13.0, L3=14.0
  Position 4: L1=17.0, L2=17.0, L3=17.0

Lattice STRUCTURAL similarity (cardinality correlation):
  L1 ↔ L2: 0.9736 (should be ~1.0 - identical structure)
  L1 ↔ L3: 0.9963
  L2 ↔ L3: 0.9797

Token overlap between L1 and L2 (position 0):
  Intersection cardinality: 0.0 (≈0 = no overlap)

KEY INSIGHT:
• Lattices have IDENTICAL structure (cardinality patterns)
• Lattices have ZERO token overlap (different content)
• ENTANGLEMENT = structural similarity, NOT content overlap
• This is the foundation of cross-modal comparison


## 7. Contrast: HLLSet Overlap vs Lattice Entanglement

The old `validate_entanglement` method compares HLLSets directly (overlap/intersection).
This is NOT true entanglement—it's just checking if sets share elements.

True entanglement compares **structures**, not content.

In [8]:
# Contrast: The OLD method checks HLLSet overlap (NOT true entanglement)
# Create HLLSets with shared base tokens (they OVERLAP)
base_tokens = [f'common_{i}' for i in range(50)]

overlapping_hllsets = [
    kernel.absorb(base_tokens + [f'unique_a_{i}' for i in range(10)]),
    kernel.absorb(base_tokens + [f'unique_b_{i}' for i in range(10)]),
    kernel.absorb(base_tokens + [f'unique_c_{i}' for i in range(10)])
]

print("HLLSet Overlap Check (OLD method - NOT true entanglement):")
has_overlap, overlap_coherence = kernel.validate_entanglement(overlapping_hllsets, epsilon=0.05)
print(f"  Has overlap: {has_overlap}")
print(f"  Overlap coherence: {overlap_coherence:.2%}")
print(f"  (This just means they share tokens - it's NOT structural entanglement)")

# Now compare: the lattices from cell 6 have NO token overlap but ARE structurally entangled
# The HLLSets above OVERLAP in tokens but that's NOT the same as entanglement

print(f"\n" + "="*60)
print("SUMMARY: Overlap ≠ Entanglement")
print("="*60)
print(f"• HLLSet overlap: checks if sets share elements")
print(f"• Lattice entanglement: checks if STRUCTURES match")
print(f"• Two lattices can be entangled with ZERO token overlap")
print(f"• Token overlap is necessary for isomorphism, not entanglement")

HLLSet Overlap Check (OLD method - NOT true entanglement):
  Has overlap: True
  Overlap coherence: 69.93%
  (This just means they share tokens - it's NOT structural entanglement)

SUMMARY: Overlap ≠ Entanglement
• HLLSet overlap: checks if sets share elements
• Lattice entanglement: checks if STRUCTURES match
• Two lattices can be entangled with ZERO token overlap
• Token overlap is necessary for isomorphism, not entanglement


## 8. Reproduction with Mutation

Create a reproduction of an HLLSet with slight mutation.

In [9]:
# Original HLLSet
original = kernel.absorb([f'data_{i}' for i in range(50)])

print(f"Original: {original.short_name}..., |A|≈{original.cardinality():.0f}")

# Reproduce (with potential mutation)
mutation_rate = 0.1  # 10% mutation
reproduced = kernel.reproduce(original, mutation_rate=mutation_rate)

print(f"Reproduced: {reproduced.short_name}..., |A|≈{reproduced.cardinality():.0f}")

# Check similarity
sim = kernel.similarity(original, reproduced)
print(f"\nSimilarity: {sim:.2%}")
print(f"  (Mutation rate {mutation_rate:.0%} affects similarity)")

Original: 1d01dd0c..., |A|≈52
Reproduced: 1d01dd0c..., |A|≈52

Similarity: 100.00%
  (Mutation rate 10% affects similarity)


## 9. Commitment (Stabilization)

Stabilize an HLLSet (in this system, HLLSets are already immutable).

In [10]:
# HLLSet to commit
uncommitted = kernel.absorb(['pending', 'data', 'tokens'])

print(f"Uncommitted: {uncommitted.short_name}...")

# Commit (stabilize)
committed = kernel.commit(uncommitted)

print(f"Committed: {committed.short_name}...")
print(f"\nSame content: {uncommitted.name == committed.name}")
print("  (HLLSets are already immutable, commit is identity operation)")

Uncommitted: fd77d4b7...
Committed: fd77d4b7...

Same content: True
  (HLLSets are already immutable, commit is identity operation)


## 10. Network Coherence

Measure coherence across a network of HLLSets.

In [11]:
# Create network of installations
import numpy as np

network = []
num_installations = 5

# Shared base + unique additions
shared_base = [f'shared_{i}' for i in range(30)]

for i in range(num_installations):
    unique = [f'unique_{i}_{j}' for j in range(20)]
    installation = kernel.absorb(shared_base + unique)
    network.append(installation)

print(f"Created network with {len(network)} installations")

# Measure pairwise similarities
similarities = []
for i in range(len(network)):
    for j in range(i+1, len(network)):
        sim = kernel.similarity(network[i], network[j])
        similarities.append(sim)

avg_coherence = np.mean(similarities)
print(f"\nNetwork coherence: {avg_coherence:.2%}")
print(f"  Min similarity: {min(similarities):.2%}")
print(f"  Max similarity: {max(similarities):.2%}")
print(f"  Std deviation: {np.std(similarities):.2%}")

Created network with 5 installations

Network coherence: 42.64%
  Min similarity: 39.73%
  Max similarity: 45.31%
  Std deviation: 1.56%


## 11. Composability

Morphisms compose naturally - chaining operations.

In [12]:
# Start with tokens
tokens = ['initial', 'data', 'set']

# Chain of transformations
result = (
    kernel.absorb(tokens)
    .union(kernel.absorb(['additional', 'tokens']))
)

# Add more
result = kernel.union(result, kernel.absorb(['even', 'more']))

print(f"Final result: {result}")
print(f"Cardinality: {result.cardinality():.2f}")
print(f"\nMorphisms compose:")
print("  absorb → union → union")
print("  Each step returns new HLLSet")
print("  Pure functional composition")

Final result: HLLSet(a55e4d50..., |A|≈8.0, backend=C/Cython)
Cardinality: 8.00

Morphisms compose:
  absorb → union → union
  Each step returns new HLLSet
  Pure functional composition


## Summary

**Kernel Capabilities**:

**Level 1: Basic Operations**
- absorb, union, intersection, difference
- Pure, stateless transformations

**Level 2: Entanglement (LATTICE-based)**
- **find_lattice_isomorphism**: Detect STRUCTURAL similarity between lattices
- **validate_lattice_entanglement**: Check if lattice STRUCTURES are mutually coherent
- find_isomorphism: HLLSet-level overlap (NOT true entanglement)
- reproduce: Create variations
- commit: Stabilize (identity in this system)

**Level 3: Network**
- Fold operations: Aggregate multiple HLLSets
- Coherence measurement: Network similarity
- Composability: Chain operations

**Key Insight: Entanglement ≠ Overlap**
- **Entanglement** is between **LATTICES** (comparing structures)
- **Overlap** is between **HLLSets** (comparing content)
- Nodes are IRRELEVANT—only lattice topology matters
- Two lattices can be entangled with ZERO token overlap

**Design Principles**:
- Stateless: No storage, no history
- Pure: Same input → same output
- Immutable: Operations return new instances
- Content-addressed: All outputs named by hash

**Next**: See `05_manifold_os.ipynb` for OS-level orchestration.